In [1]:
from __future__ import print_function, division
import cv2 as cv2
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Type, Dict, Any
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
from tqdm import tqdm
import os.path
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import os
from threading import Thread
from queue import Empty, Queue
import threading
from sklearn.utils import shuffle
import random
import accimage

In [2]:
torch.manual_seed(378) # обнулили генератор рандома у торча
np.random.seed(378)# обнулили генератор рандома у нампая
torch.backends.cudnn.deterministic = True # ждем одинаковый результат у GPU и CPU Для одинаковых данны переданных торчу
torch.backends.cudnn.benchmark = False # вроде аналогично строчке выше
ia.seed(1)

In [3]:
torch.cuda.device_count()
device1 = torch . device ( "cuda:0" if torch . cuda . is_available () else "cpu" )
print(device1)
device2 = torch.device('cpu')

cuda:0


In [4]:
def start_train(model): # запускаем обучение всех слоев
    for param in model.parameters(): 
        param.requires_grad = True
def draw_circle(image, landmarks): # дорисовываем окружность на изображении и возвращаем обратно
    imageWithCircle = cv2.circle(image, (int(landmarks[0][0]),int(landmarks[0][1]) ),int(landmarks[0][2]), (255, 0 , 0),  10)
    return imageWithCircle


def show_landmarks(image, landmarks): # конкретно эта функция, скорее всего не понадобится
    """Show image with landmarks"""
    plt.imshow(draw_circle(image,landmarks) )
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r') # начало координат не совпадает , надо учесть
    # для картинки надо, для нейросетки не нужно, начало координат должно совпасть
    # в numpy 0-левая координата - у, первая - иксы(столбцы)
    # порисовать диск солнце порисовать кружочки через opencv(там с координатами тоже не все гладко, надо внимательно смотреть)
    # размеры реальных снимков 1920х1920 а не 1200х1200, надо как-то или к долям 1 или сразу в 1920х1920 формат
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def calculate_loss(model_result : torch.tensor, 
                   data_target  : torch.tensor,
                   loss_function: torch.nn.Module = torch.nn.MSELoss()): #reduction = None
    lossXY =  (loss_function(model_result[:,:2], data_target[:,:2]))**(0.5) # тут из батчей получаю, править
    lossR =  loss_function(model_result[:,2], data_target[:,2]) # потом корень извлечь
    #return {'lossXY': lossXY.item(),'lossR': lossR.item() }
    return { lossXY.item(), lossR.item() }


In [5]:
alexnet = models.alexnet(pretrained=True, progress=False)
for p in alexnet.parameters():
    print(p.device)
    break
for param in alexnet.parameters(): # запрещаем обучаться alexnet
    param.requires_grad = False
alexnet.fc = torch.nn.Sequential(nn.Linear(in_features=2048, out_features=256, bias=True),
                            nn.ReLU(),
                            nn.Linear(in_features=256, out_features=128, bias=True),
                            nn.ReLU(), # SeLU или GeLU возможно подойдут лучше, надо поиграться
                            nn.Linear(in_features=128, out_features=3, bias=True)) # мой франкенштейн для регрессии, который подцепили к resnet сфот
_ = alexnet.to(device1)

cpu


In [6]:
resnet152 = models.resnet152(pretrained=True, progress=False)
for p in resnet152.parameters():
    print(p.device)
    break
for param in resnet152.parameters(): # запрещаем обучаться resnet 152
    param.requires_grad = False
resnet152.fc = torch.nn.Sequential(nn.Linear(in_features=2048, out_features=256, bias=True),
                            nn.ReLU(),
                            nn.Linear(in_features=256, out_features=128, bias=True),
                            nn.ReLU(), # SeLU или GeLU возможно подойдут лучше, надо поиграться
                            nn.Linear(in_features=128, out_features=3, bias=True)) # мой франкенштейн для регрессии, который подцепили к resnet сфот
_ = resnet152.to(device1)

cpu


# упомянуть челиков из imgaug
## тут агментация данных дальше

In [7]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),# horizontally flip 20% of the images flipud
    iaa.Flipud(0.5),# vertically flip 20% of the images flipud добавить на 180 градусов поворот
    iaa.GaussianBlur(sigma=(0, 5)), # blur images with a sigma of 0 to 0.5
    #iaa.Invert(0.1) # пока уберем
    iaa.Affine(rotate=(-180,180))# поворот на 180 через афииные преобразования
])
# kps = KeypointsOnImage([
#     Keypoint(x=landmarks[:,0], y=landmarks[:,1])
# ], shape=(1920,1920,3))

In [8]:
def find_files(dataset_dir):
    image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir)]
    return image_filenames


class thread_killer(object):    
    """Boolean object for signaling a worker thread to terminate"""
    def __init__(self):
        self.to_kill = False

    def __call__(self):
        return self.to_kill

    def set_tokill(self, tokill):
        self.to_kill = tokill


def threaded_batches_feeder(tokill, batches_queue, dataset_generator):
    while tokill() == False:
        for img, landmarks in dataset_generator:
            batches_queue.put((img, landmarks), block=True)
            if tokill() == True:
                return


def threaded_cuda_batches(tokill,cuda_batches_queue,batches_queue):
    while tokill() == False:
        (img, landmarks) = batches_queue.get(block=True)
        img = torch.from_numpy(img)
        landmarks = torch.from_numpy(landmarks)       
        
        img = Variable(img.float()).to(device1)
        landmarks = Variable(landmarks.float()).view(-1,3).to(device1)
        
        cuda_batches_queue.put((img, landmarks), block=True)
        
        if tokill() == True:
            return


class threadsafe_iter:
    """
    Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.it)

def get_objects_i(objects_count):
    """Cyclic generator of paths indices
    """
    current_objects_id = 0
    while True:
        yield current_objects_id
        current_objects_id  = (current_objects_id + 1) % objects_count

In [9]:

# for item, url in enumerate(urls):
#     name = "Поток %s" % (item+1)
#     thread = DownloadThread(url, name)
#     thread.start()
 

In [10]:
class SunLandmarksDataset(Dataset):
    """Sun Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform = None, batch_size = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file,delimiter=',')
        self.root_dir = root_dir
        self.batch_size = batch_size
        
        self.objects_id_generator = threadsafe_iter(get_objects_i(self.landmarks_frame.shape[0]))
        
        self.lock = threading.Lock()  # mutex for input path
        self.yield_lock = threading.Lock()  # mutex for generator yielding of batch
        self.init_count = 0
        self.cache = {}

    def __len__(self):
        return len(self.landmarks_frame)

    def shuffle(self):
        self.landmarks_frame = shuffle(self.landmarks_frame)
    
    def __iter__(self):
        while True:
            with self.lock:
                if (self.init_count == 0):
                    self.shuffle()
                    self.imgs = []
                    self.landmarks = []
                    self.init_count = 1
            
            for obj_id in self.objects_id_generator:
                img_name = os.path.join(self.root_dir , self.landmarks_frame.iloc[obj_id, 0])
                if img_name in self.cache:
                    img = self.cache[img_name]
                else:
                    img = accimage.Image(img_name)
                    image_np = np.empty([img.channels, img.height, img.width], dtype=np.uint8)
                    img.copyto(image_np)
                    self.cache[img_name] = img
                    
                img = np.transpose(image_np, (1, 2, 0))
                
                landmarks = self.landmarks_frame.iloc[obj_id, 1:]
                landmarks = np.array([landmarks])
                landmarks = landmarks.astype('float').reshape(1, 3)
                
                kps = KeypointsOnImage([
                        Keypoint(x=landmarks[0,0]*1920, y=landmarks[0,1]*1920) # домножил, тк нормировал
                ], shape=img.shape)
                image_aug, landmarks_after =seq(image = img, keypoints = kps) # тут применяю аугментацию к фотке и целевой переменной
                landmarks[0,0] = landmarks_after.keypoints[0].x/1920 # обратно отнормировал 
                landmarks[0,1] = landmarks_after.keypoints[0].y/1920
                img = image_aug
                
                img = img.transpose((2, 0, 1))[np.newaxis,...] # для тензоров
#                 landmarks = self.landmarks_frame.iloc[obj_id, 1:]
#                 landmarks = np.array([landmarks])
                landmarks = landmarks[np.newaxis,...]
                
                # Concurrent access by multiple threads to the lists below
                with self.yield_lock:
                    if (len(self.imgs)) < self.batch_size:
                        self.imgs.append(img)
                        self.landmarks.append(landmarks)
                    if len(self.imgs) % self.batch_size == 0:
                        self.imgs = np.concatenate(self.imgs, axis=0)
                        self.landmarks = np.concatenate(self.landmarks, axis=0)
                        yield (self.imgs, self.landmarks)
                        self.imgs, self.landmarks = [], []
                        
            # At the end of an epoch we re-init data-structures
            with self.lock:
                self.landmarks_frame = shuffle(self.landmarks_frame)
                self.init_count = 0

In [11]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        image = image.copy() # какая-та жопа была с шагом, текст ошибки прямо советовал просто копировать
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}

# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""
#     def __call__(self, sample):
#         image = sample
#         return torch.from_numpy(image)

In [12]:
batch_size = 10
sun_dataset_train = SunLandmarksDataset(csv_file='sun_disk_pos_database01train.csv',
                                           root_dir='/app/images',
                                           transform=transforms.Compose([
                                               ToTensor()
                                           ]), batch_size=batch_size)
sun_dataset_test = SunLandmarksDataset(csv_file='sun_disk_pos_database01test.csv',
                                           root_dir='/app/images',
                                           transform=transforms.Compose([
                                               ToTensor()
                                           ]), batch_size=batch_size)

In [13]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.MSELoss(), # loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss() 
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                lr_scheduler_class: Any = torch.optim.lr_scheduler.CosineAnnealingLR,
                initial_lr = float,
                lr_scheduler_params: Dict = {},
                batch_size = int,
                max_epochs = 100,
                early_stopping_patience = 20):

    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)

#     train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size,drop_last=False)
#     val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,drop_last=False)
    
    best_val_loss = None
    best_epoch = None
    loss_value = []
    loss_valueXY = []
    loss_valueR = []
    
    batches_queue_length =16

    train_batches_queue = Queue(maxsize=batches_queue_length)
    train_cuda_batches_queue = Queue(maxsize=8)
    train_thread_killer = thread_killer()
    train_thread_killer.set_tokill(False)
    preprocess_workers = 16

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, train_batches_queue, train_dataset))
        thr.start()

    train_cuda_transfers_thread_killer = thread_killer()
    train_cuda_transfers_thread_killer.set_tokill(False)
    train_cudathread = Thread(target=threaded_cuda_batches, args=(train_cuda_transfers_thread_killer, train_cuda_batches_queue,
                                                                  train_batches_queue))
    train_cudathread.start()
    

    test_batches_queue = Queue(maxsize=batches_queue_length)
    test_cuda_batches_queue = Queue(maxsize=8)
    test_thread_killer = thread_killer()
    test_thread_killer.set_tokill(False)
    preprocess_workers = 16

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, test_batches_queue, sun_dataset_test))
        thr.start()

    test_cuda_transfers_thread_killer = thread_killer()
    test_cuda_transfers_thread_killer.set_tokill(False)
    test_cudathread = Thread(target=threaded_cuda_batches, args=(test_cuda_transfers_thread_killer, test_cuda_batches_queue,
                                                                  test_batches_queue))
    test_cudathread.start()
    
    
    Steps_Per_Epoch_Train = len(train_dataset)/batch_size # количество итераций обучения и валидации
    Steps_Per_Epoch_Test = len(val_dataset)/batch_size
    
    
    for epoch in range(max_epochs):
        
        print(f'Epoch {epoch}')
        train_single_epoch(model, optimizer, loss_function, train_cuda_batches_queue,Steps_Per_Epoch_Train) # передать х
        val_metrics, lossXY, lossR = validate_single_epoch(model, loss_function, test_cuda_batches_queue,Steps_Per_Epoch_Test)
        loss_value = np.append(loss_value,val_metrics) # добавляю в массив значение функци потерь
        loss_valueXY = np.append(loss_valueXY,lossXY)
        loss_valueR = np.append(loss_valueR,lossR)
        np.save('loss_value+aug+proc',loss_value )
        np.save('loss_valueXY+aug+proc',loss_valueXY )
        np.save('loss_valueR+aug+proc',loss_valueR )
        print(f'Validation metrics: \n{val_metrics}')
        


        lr_scheduler.step() #lr_scheduler.step(val_metrics['loss'])


        
        if best_val_loss is None or best_val_loss > val_metrics:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics
            best_epoch = epoch
            torch.save(model, 'saved_models/best_model+aug+proc.pth') 

        
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return
        
        
    val_thread_killer.set_tokill(True) # убиваю потокои, так же убить валидационные
    val_cuda_transfers_thread_killer.set_tokill(True)
    for _ in range(preprocess_workers):
        try:
            # Enforcing thread shutdown
            val_batches_queue.get(block=True, timeout=1)
            val_cuda_batches_queue.get(block=True, timeout=1)
        except Empty:
            pass

In [14]:
def validate_single_epoch(model: torch.nn.Module,
                        loss_function: torch.nn.Module, 
                        cuda_batches_queue: Queue, 
                        Per_Step_Epoch:int):
    



    test_loss = 0
    correct = 0
    lossXY = 0
    lossR = 0
    loss1 = 0
    loss2 = 0
    model.eval()
    for batch_idx in range(int(Per_Step_Epoch)): # тут продумать 
        data_image, target = cuda_batches_queue.get(block=True)
        data_out = model(data_image)
        loss = loss_function(data_out, target) # посчитали на изображении
        test_loss += loss.item() # прибавили
        loss1 , loss2 = calculate_loss(data_out, target) # тут считаю свои метрики
        lossXY +=loss1
        lossR  +=loss2
        
    test_loss /= Per_Step_Epoch*batch_size
    lossXY    /= Per_Step_Epoch*batch_size # корень извлечь и поправить функцию расчета
    lossR     /= Per_Step_Epoch*batch_size  # корень извлечь
    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #      test_loss, correct, len(data_loader.dataset),
    #      100. * correct / len(data_loader.dataset)))
    #return {'loss':test_loss,'lossXY':lossXY,'lossR':lossR, } #, correct
    #return {'loss':test_loss,'lossXY':lossXY,'lossR':lossR}
    return {test_loss,lossXY,lossR}

In [15]:
def train_single_epoch(model: torch.nn.Module,
                    optimizer: torch.optim.Optimizer, 
                    loss_function: torch.nn.Module,
                    cuda_batches_queue: Queue, 
                    Per_Step_Epoch:int):
                    #data_loader: torch.utils.data.DataLoader):

    model.train()
    pbar = tqdm(total=Per_Step_Epoch)
    for batch_idx in range(int(Per_Step_Epoch)): # тут продумать 
        data_image, target = cuda_batches_queue.get(block=True)
        print(data_image.size())
        target = Variable(target).view(-1,3) # трансформировали переменные для pytorch , volatile=True
        target = target.to(device1)
        optimizer.zero_grad() # обнулили\перезапустии градиенты для обратного распространения
        data_out = model(data_image) # применили модель к данным
        loss = loss_function(data_out, target) # применили фуннкцию потерь
        loss.backward() # пошли по графу нейросетки обратно
        optimizer.step()# выполняем наш градиентный спуск по вычисленным шагам в предыдущей строчке
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})
        
        
        
    return {'loss': loss.item()}

In [16]:
    train_model(resnet152, 
            train_dataset = sun_dataset_train, 
            val_dataset =  sun_dataset_test,  
            loss_function = torch.nn.MSELoss(), 
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingLR,
            lr_scheduler_params = {'T_max': 100, 'last_epoch': -1, 'eta_min': 1e-7},
            initial_lr=1e-4,
            batch_size = 10)

  0%|          | 0/214.1 [00:00<?, ?it/s]

Epoch 0
<built-in method size of Tensor object at 0x7f7930164d38>


  0%|          | 1/214.1 [00:26<1:35:20, 26.84s/it, loss=0.179]

<built-in method size of Tensor object at 0x7f793016c630>


RuntimeError: Unable to find a valid cuDNN algorithm to run convolution